In [ ]:
# titanv 1:
# screen -S dbert-cache1
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/twinews-logs
# DOCKER_PORT=9963 nn -o nohup-dbert-cache-$HOSTNAME-1.out ~/docker/keras/run-jupython.sh ~/notebooks/twinews/hjnotebooks/dbert-cache.ipynb titanv
# observe ~/twinews-logs/nohup-dbert-cache-$HOSTNAME-1.out

## Imports

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = isNotebook # False, True, isNotebook

In [ ]:
import os
from systemtools.hayj import *
if not labia():
    os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
import logging
import math
import tensorflow as tf
from tensorflow.keras import callbacks
from transformers import \
(
    DistilBertConfig,
    DistilBertTokenizer,
    TFDistilBertForSequenceClassification,
)

In [ ]:
from systemtools.basics import *
from systemtools.printer import *
from systemtools.file import *
from systemtools.location import *
from datatools.jsonutils import *
from machinelearning.encoder import *
from newssource.asa2.utils import *
from machinelearning import tf2utils

In [ ]:
from newssource.dbert.utils import *
from twinews.utils import *
from twinews.models.ranking import *

## Config

In [ ]:
logger = Logger('dbert-cache.log')

In [ ]:
# The best model is dbert-94bef_ep32-distilbert-mean
modelNames = ['94bef_ep32', None]
bp(modelNames, 5, logger)

In [ ]:
maxLength = 512
batchSize = 16

In [ ]:
# layers = ['distilbert', 'pre_classifier', 'dropout', 'classifier'] if modelPath is not None else ['distilbert']
layer = 'distilbert'

In [ ]:
(user, password, host) = getMongoAuth(user='hayj')

In [ ]:
newsCollection = getNewsCollection(logger=logger)

In [ ]:
ids = newsCollection.distinct('_id')

In [ ]:
if TEST:
    ids = ids[:3]

In [ ]:
bp(ids, logger)

## Caching

In [ ]:
pbar = ProgressBar(len(ids) * len(modelNames), logger=logger, printRatio=0.0001)

In [ ]:
for modelName in modelNames:
    if modelName is None:
        modelPath = None
        modelName = 'base'
    else:
        if lri():
            modelPath = sortedGlob(nosaveDir() + "/dbert-tmp/" + modelName + "/epochs/ep*")[0]
        else:
            modelPath = sortedGlob(homeDir() + "/asa/dbert-tmp/" + modelName + "/epochs/ep*")[0]
    # We init the model:
    if modelPath is None:
        dbertConfig = DistilBertConfig.from_pretrained\
        (
            "distilbert-base-uncased",
            num_labels=1200,
            max_length=maxLength,
        )
        model = TFDistilBertForSequenceClassification.from_pretrained\
        (
            "distilbert-base-uncased",
            config=dbertConfig,
        )
    else:
        dbertConfig = DistilBertConfig.from_pretrained(modelPath + '/config.json')
        model = TFDistilBertForSequenceClassification.from_pretrained\
        (
            modelPath + '/tf_model.h5',
            config=dbertConfig,
        )
    log(modelName + " dbert model loaded.", logger)
    # We init the cache:
    cache = SerializableDict\
    (
        "twinews-dbert-" + modelName,
        user=user, host=host, password=password,
        useMongodb=True, logger=logger,
    )
    # We infer all embeddings for each row in each file:
    for currentId in ids:
        row = newsCollection.findOne({'_id': currentId}, projection={'detokText': True})
        if dictContains(row, 'detokText'):
            text  = row['detokText']
            currentHash = objectToHash(text)
            if currentHash not in cache:
                encodedText = tf2utils.distilBertEncode\
                (
                    text,
                    maxLength=maxLength,
                    multiSamplage=True,
                    preventTokenizerWarnings=True,
                )
                encodedBatches = chunks(encodedText, batchSize)
                embeddings = []
                for encodedBatch in encodedBatches:
                    outputs = tf2utils.getDistilBertRepresentations(model, np.array(encodedBatch), layer=layer)
                    for output in outputs:
                        embeddings.append(np.array(output))
                embeddings = np.mean(embeddings, axis=0)
                cache[currentHash] = embeddings
        pbar.tic()